Factuality with LLMs.  

Stephan Raaijmakers, LUCL, 01.04.2025

In [ ]:
!pip install langchain_huggingface

In [2]:
from transformers import AutoModelForCausalLM , AutoTokenizer, pipeline
import sys
from tqdm import tqdm
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import os
import re

In [3]:
from langchain_core.messages import (
    HumanMessage,
    SystemMessage,
)

In [4]:
from langchain_huggingface.llms import HuggingFacePipeline

In [5]:
from langchain_huggingface import ChatHuggingFace,HuggingFaceEndpoint

In [6]:
from google.colab import userdata
HUGGINGFACE_TOKEN=userdata.get('HF_TOKEN')

Notice you will need a Huggingface PRO account ($9/mo) for the InferenceClient on non-free tier models.

In [7]:
llm = HuggingFaceEndpoint(
    #repo_id="HuggingFaceH4/zephyr-7b-beta",
    repo_id="mistralai/Mistral-7B-Instruct-v0.2",
    task="text-generation",
    max_new_tokens=512,
    do_sample=False,
    repetition_penalty=1.03,
    temperature=0.0
)

In [22]:
def check_facts(lines, output_file):
    print("Checking facts...")
    #HUGGINGFACE_TOKEN=os.environ["HUGGINGFACE_TOKEN"] # run first: export HUGGINGFACE_TOKEN="..." in shell
    outp=open(output_file,"w")

    chat_model = ChatHuggingFace(llm=llm)

    for i in tqdm(range(len(lines))):
        sentence=' '.join(lines[i])

        messages = [
        SystemMessage(content="Check the factuality of the following sentence. Look for the facts, check them and then report with True or False for the entire sentence. Explain your verdict."),
        HumanMessage(
        content="Sentence:"+sentence
          ),
        ]

        completion = chat_model.invoke(messages)

        fact_checked=completion.content
        fact_checkeded=re.sub("\n"," ",fact_checked)
        fact_checked=re.sub("Here is the output: ","",fact_checked)
        outp.write("Sentence:%s\nVerdict:%s\n"%(sentence,fact_checked))

In [ ]:
text="""
Rembrandt was a German painter.
Trump is a Democrat.
Picasso is a cubist painter.
Melanzane is a vegetable.
Ice is lighter than water.
You can approach pi by throwing darts in a circle.
"""

In [28]:
def main_file(input_file, output_file):
    with open(input_file,"r") as f:
        lines = [z for z in [x.rstrip().split(" ") for x in f.readlines()]]
    check_facts(lines, output_file)
    print("See ", output_file)

In [ ]:
def main(output_file):
    lines=text.split("\n")
    check_facts(lines, output_file)
    print("See ", output_file)

In [ ]:
main_file("facts.txt", "fact_checked.out")

In [ ]:
main("fact_checked.out")